In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import os


In [ ]:
Carga_Datos = pd.read_csv(r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\CSV_Salida.csv")

### Datos temporales para pruebas 


In [ ]:
# datos para las pruebas:
# data = {'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 130000000.0, 110000000.0, 110000000.0, 110000000.0, 110000000.0, 170000000.0, 170000000.0, 170000000.0, 150000000.0, 150000000.0, 190000000.0, 190000000.0, 190000000.0, 90000000.0, 90000000.0, 90000000.0, 90000000.0],
#         'dB': [50, 25, 36, 26, 27, 26, 98, 27, 23, 46, 93, 68, 126, 30, 129, 119, 69, 114, 89, 69, 63, 27, 96, 78]}

data = {
    'Frecuencia (Hz)': [90000000.0, 90000000.0, 90000000.0, 110000000.0, 110000000.0,
                         110000000.0, 110000000.0, 130000000.0, 130000000.0, 130000000.0,
                         130000000.0, 130000000.0, 150000000.0, 150000000.0, 170000000.0,
                         170000000.0, 170000000.0, 190000000.0, 190000000.0, 190000000.0,
                         90000000.0, 90000000.0, 90000000.0, 90000000.0, 110000000.0, 110000000.0,
                         130000000.0, 150000000.0, 150000000.0,170000000.0, 190000000.0, 190000000.0, 90000000.0,90000000.0,90000000.0],
    'dB': [50, 25, 36, 26, 27, 26, 98, 27, 23, 46, 93, 68, 126, 30, 129, 119, 69, 114, 89, 69, 63, 27, 96, 78, 98,77,88,99, 27,-11,-22,-33,-5555,-1111,-2222]
}

Carga_Datos = pd.DataFrame(data)
print(Carga_Datos)

###  Borra los primeros datos con caracteres vacíos o nulos  

|Frecuencia (Hz)|        dB        |
|---------------|------------------|
|    nan        |-101.9620361328125|


In [ ]:
Carga_Datos= Carga_Datos.dropna(subset=['Frecuencia (Hz)'])
# print(Carga_Datos['Frecuencia (Hz)'].mean())

### Cantidad total de datos por cada frecuencia     


In [ ]:
conteo_frecuencia = Carga_Datos['Frecuencia (Hz)'].value_counts()
print(conteo_frecuencia)

### Separación de muestras por cada frecuencia  

In [ ]:
recuentos_por_muestra = {}

# Convertir la columna 'Frecuencia (Hz)' en un arreglo NumPy
frecuencias = Carga_Datos['Frecuencia (Hz)'].values

# Inicializar variables para llevar el seguimiento de la muestra actual y el conteo actual
muestra_actual = None
conteo_actual = 0

for frecuencia in frecuencias:
    if muestra_actual is None:
        muestra_actual = frecuencia
        conteo_actual = 1
    elif muestra_actual == frecuencia:
        conteo_actual += 1
    else:
        if muestra_actual not in recuentos_por_muestra:
            recuentos_por_muestra[muestra_actual] = []
        recuentos_por_muestra[muestra_actual].append(conteo_actual)
        
        muestra_actual = frecuencia
        conteo_actual = 1

# Agregar el último conjunto de conteos a la salida
if muestra_actual is not None:
    if muestra_actual not in recuentos_por_muestra:
        recuentos_por_muestra[muestra_actual] = []
    recuentos_por_muestra[muestra_actual].append(conteo_actual)

for muestra, conteos in recuentos_por_muestra.items():
    print(f'Frecuencia: {muestra} - Conteos: {", ".join(map(str, conteos))}')

#  Tratado de datos 

### --------------------------------------------------------------------------------------------------------------------------------------------------------

Para poder tratar los datos se dividen por grupos esos hacen referencia a las muestras que se toman, para posteriormente sacar el promedio de estos y el los máximos 

In [ ]:
def dividir_y_guardar_grupos(Carga_Datos, carpeta_salida):
    grupos = []
    grupo_actual = None
    cambios = np.where(Carga_Datos['Frecuencia (Hz)'].values[:-1] > Carga_Datos['Frecuencia (Hz)'].values[1:])[0]

    inicio = 0
    for cambio in cambios:
        grupo_actual = Carga_Datos.iloc[inicio:cambio + 1]
        grupos.append(grupo_actual)
        inicio = cambio + 1

    grupo_actual = Carga_Datos.iloc[inicio:]
    grupos.append(grupo_actual)

    # Crear la carpeta de salida si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    # Guardar los grupos en archivos CSV
    for i, grupo in enumerate(grupos):
        nombre_archivo = os.path.join(carpeta_salida, f'Muestra_{i + 1}.csv')
        grupo.to_csv(nombre_archivo, index=False)
        print(f"Grupo {i + 1} guardado en '{nombre_archivo}'")

carpeta_muestras = 'Muestras'
dividir_y_guardar_grupos(Carga_Datos, carpeta_muestras)


# Tratado de las muestras 

Asignación de las frecuencias para cada muestra, teniendo en cuenta el ancho de banda 

Frec_max-Frec_min/cont_frec-1


In [ ]:
carpeta = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras'  
archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

BW_TEMP = 0
MST_ANT = False

# Procesar cada archivo CSV en la carpeta
for archivo_csv in archivos_csv:
    archivo_path = os.path.join(carpeta, archivo_csv)
    data = pd.read_csv(archivo_path)

    # Identificar las secuencias de frecuencias iguales
    secuencias = data['Frecuencia (Hz)'].unique()

    # Calcular el valor de BW
    if len(secuencias) >= 2: # REVISA SI HAY OTRA FRECUENCIA 
        BW = (secuencias[1] - secuencias[0])
        BW_TEMP = BW
        MST_ANT = True
        
    elif (MST_ANT == True):
        BW = BW_TEMP

    else:               # si no no hubo una muestra anterior y no tiene un valor de BW lo tomara por defecto como 20M.
        BW = 20000000

    # Crear un nuevo DataFrame para los resultados
    nuevo_df = pd.DataFrame(columns=['Frecuencia (Hz)', 'dB'])

    # Calcular y agregar los nuevos valores de frecuencia 
    for secuencia in secuencias:
        num_samples = (data['Frecuencia (Hz)'] == secuencia).sum()
        
        min_value = secuencia - (BW / 2)
        max_value = secuencia + (BW / 2)

        # Asegurar que num_samples sea al menos 2
        if num_samples == 1:
            num_samples = 2

        incremento_frec = BW / (num_samples - 1)
        nuevos_valores_frecuencia = min_value + np.arange(num_samples) * incremento_frec
        nuevos_valores_frecuencia = pd.Series(nuevos_valores_frecuencia)
        nuevos_valores_frecuencia.name = 'Frecuencia (Hz)'
        nuevo_df = pd.concat([nuevo_df, pd.concat([nuevos_valores_frecuencia, data[data['Frecuencia (Hz)'] == secuencia]['dB'].reset_index(drop=True)], axis=1)], ignore_index=True)

    # Ordenar el nuevo DataFrame por frecuencia
    nuevo_df.sort_values(by=['Frecuencia (Hz)'], inplace=True)

    # Reiniciar los índices del nuevo DataFrame
    nuevo_df.reset_index(drop=True, inplace=True)

    # Sobreescribir el archivo CSV original con los nuevos datos
    nuevo_df.to_csv(archivo_path, index=False)

    print(f'Archivo procesado y sobrescrito: {archivo_path}')


# Filtrado de datos 

Promedio y máximos unificado de todas las muestras 

In [ ]:

# Ruta de la carpeta de muestras
carpeta_archivos = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras'

# Obtiene la ruta del directorio anterior
directorio= os.path.dirname(carpeta_archivos)

# Carpeta para guardar los resultados
carpeta_resultados = os.path.join(directorio, 'Resultados')

# Verifica si la carpeta ya existe, y si no, créala
if not os.path.exists(carpeta_resultados):
    os.makedirs(carpeta_resultados)

# Tolerancia para realizar el análisis de los datos 
tolerancia = 1000

# Función para procesar un archivo CSV y obtener el promedio de dB dentro de la tolerancia
def procesar_archivo(archivo):
    data = pd.read_csv(archivo)
    frecuencia = data['Frecuencia (Hz)'].values
    db = data['dB'].values
    promedios = []
    maximos = []
    

    inicio = 0
    fin = 0
    
    for i in range(len(frecuencia)):
        if frecuencia[i] <= frecuencia[inicio] + tolerancia:
            fin = i
        else:
            # Calcula el promedio y máximo dentro de la tolerancia
            promedio = np.mean(db[inicio:fin+1])
            maximo = np.max(db[inicio:fin+1])
            promedios.append([frecuencia[inicio], promedio])
            maximos.append([frecuencia[inicio], maximo])
            
            # Actualiza el inicio y fin para el siguiente rango
            inicio = i
            fin = i
    
    # Agrega el último rango
    promedio = np.mean(db[inicio:])
    maximo = np.max(db[inicio:])
    promedios.append([frecuencia[inicio], promedio])
    maximos.append([frecuencia[inicio], maximo])
    
    return pd.DataFrame(promedios, columns=['Frecuencia (Hz)', 'Promedio (dB)']), pd.DataFrame(maximos, columns=['Frecuencia (Hz)', 'Máximo (dB)'])

# Obtiene la lista de archivos en la carpeta
archivos = [os.path.join(carpeta_archivos, archivo) for archivo in os.listdir(carpeta_archivos) if archivo.endswith('.csv')]
archivos.sort()

promedio_acumulado = pd.DataFrame(columns=['Frecuencia (Hz)', 'Promedio (dB)'])
maximo_acumulado = pd.DataFrame(columns=['Frecuencia (Hz)', 'Máximo (dB)'])

for archivo in archivos[:-1]:
    promedio, maximo = procesar_archivo(archivo)
    
    # Combina los resultados con el promedio acumulado
    promedio_acumulado = pd.concat([promedio_acumulado, promedio], ignore_index=True)
    maximo_acumulado = pd.concat([maximo_acumulado, maximo], ignore_index=True)

# Calcula el promedio final
promedio_final = promedio_acumulado.groupby('Frecuencia (Hz)').mean().reset_index()
maximo_final = maximo_acumulado.groupby('Frecuencia (Hz)').max().reset_index()

# Ruta para guardar los archivos en la carpeta "Resultados"
ruta_promedio_final = os.path.join(carpeta_resultados, 'promedio_final.csv')
ruta_maximo_final = os.path.join(carpeta_resultados, 'maximo_final.csv')

# Guarda los resultados en dos archivos CSV separados en la carpeta "Resultados"
promedio_final.to_csv(ruta_promedio_final, index=False)
maximo_final.to_csv(ruta_maximo_final, index=False)


Promedio de las muestras por individual

In [ ]:

carpeta_datos = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"

carpeta_resultados = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados"

carpeta_tratadas = os.path.join(carpeta_resultados, "Muestra_Promediadas") 
os.makedirs(carpeta_tratadas, exist_ok=True)

archivos_csv = [archivo for archivo in os.listdir(carpeta_datos) if archivo.endswith(".csv")]

# Función para promediar los datos en grupos 
def promediar_en_grupos(datos):
    grupos_promediados = []
    grupo_actual = []
    for i, fila in enumerate(datos):
        grupo_actual.append(fila)
        if (i + 1) % 10 == 0:
            promedio_grupo = [sum(col) / len(col) for col in zip(*grupo_actual)]
            grupos_promediados.append(promedio_grupo)
            grupo_actual = []
    if grupo_actual:
        promedio_grupo = [sum(col) / len(col) for col in zip(*grupo_actual)]
        grupos_promediados.append(promedio_grupo)
    return grupos_promediados

for archivo_csv in archivos_csv:
    # Leer el archivo CSV
    ruta_csv = os.path.join(carpeta_datos, archivo_csv)
    df = pd.read_csv(ruta_csv)

    # Promediar ambas columnas en grupos y crear un nuevo DataFrame
    grupos_promediados = promediar_en_grupos(df.values)
    df_promediado = pd.DataFrame(grupos_promediados, columns=["Promedio_Frecuencia (Hz)", "Promedio_dB"])

    # Generar el nombre del archivo filtrado
    nombre_filtrado = os.path.splitext(archivo_csv)[0] + "_filtrado.csv"
    ruta_filtrado = os.path.join(carpeta_tratadas, nombre_filtrado)

    df_promediado.to_csv(ruta_filtrado, index=False)

print("Archivos filtrados y promediados guardados en la carpeta 'Muestra_tratadas'.")

# Genera los plots de las muestras por separado 

Toma las muestras generadas y gráfica cada una de ellas por grupos de 1000 datos para que sea visibles

In [ ]:
carpeta_datos = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"

# Crear la carpeta 'plots' si no existe
carpeta_plots = os.path.join(carpeta_datos, "plots")
os.makedirs(carpeta_plots, exist_ok=True)

# Lista de archivos CSV en la carpeta de datos
archivos_csv = [archivo for archivo in os.listdir(carpeta_datos) if archivo.endswith(".csv")]

# Función para dividir los datos en grupos de 2000
def dividir_en_grupos(datos):
    grupos = []
    grupo_actual = []
    for i, fila in enumerate(datos):
        grupo_actual.append(fila)
        if (i + 1) % 1000 == 0:
            grupos.append(grupo_actual)
            grupo_actual = []
    if grupo_actual:
        grupos.append(grupo_actual)
    return grupos

# Procesar cada archivo CSV
for archivo_csv in archivos_csv:
    # Leer el archivo CSV
    ruta_csv = os.path.join(carpeta_datos, archivo_csv)
    df = pd.read_csv(ruta_csv)

    # Dividir los datos en grupos de 2000
    grupos = dividir_en_grupos(df.values)

    # Generar y guardar los gráficos para cada grupo
    for i, grupo in enumerate(grupos):
        plt.figure(figsize=(18, 8))
        x = [fila[0] for fila in grupo]
        y = [fila[1] for fila in grupo]
        plt.plot(x, y)
        plt.xlabel("Frecuencia (Hz)")
        plt.ylabel("dB")
        plt.title(f"Gráfico {i + 1} - {archivo_csv}")
        ruta_plot = os.path.join(carpeta_plots, f"{archivo_csv}_{i + 1}.png")
        plt.savefig(ruta_plot)
        plt.close()

print("Gráficos generados y guardados en la carpeta 'plots'.")

# Grafica individual 



In [ ]:
# Leer el archivo CSV con los datos
archivo_csv = r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados\promedio_final.csv'
df = pd.read_csv(archivo_csv)

# Extraer las columnas de frecuencia y promedio
frecuencia = df['Frecuencia (Hz)'].to_numpy()
promedio = df['Promedio (dB)'].to_numpy()

plt.figure(figsize=(18, 8))
plt.plot(frecuencia, promedio, marker='', linestyle='-')  

# Configurar etiquetas y título
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Promedio (dB)')
plt.title('Promedio Total por Intervalo de Frecuencia')

# Mostrar el gráfico
plt.xticks(rotation=45)
plt.grid(True)  # Agrega una cuadrícula para mayor claridad
plt.tight_layout()
plt.show()

# Setpoint en base al valor promedio genera csv con valores que max que sobrepasaron el setpoint


### máximos partiendo de setpoint para un único archivo, promedio de todas las muestras        

In [ ]:
def Setpoint_Max(archivo_csv, Setpoint,carpeta_resultados):
    data = pd.read_csv(archivo_csv)
    encabezados = data.columns
    promedio_segunda_columna = data.iloc[:, 1].mean()
    setpoint = promedio_segunda_columna + Setpoint  
    valores_superan_setpoint = data[data.iloc[:, 1] > setpoint]


    nombre_original_sin_extension = os.path.splitext(os.path.basename(archivo_csv))[0]

    nombre_salida = f"{nombre_original_sin_extension}_Setpoint_Max.csv"

    if not os.path.exists(carpeta_resultados):
        os.makedirs(carpeta_resultados)

    ruta_salida = os.path.join(carpeta_resultados, nombre_salida)
    valores_superan_setpoint.to_csv(ruta_salida, index=False)
    
    print(f"Valor promedio dB {nombre_original_sin_extension}: {promedio_segunda_columna} dB")
    print(f"Valor Setpoint para {nombre_original_sin_extension}: {setpoint} dB")
    print(f"El archivo de resultados se ha guardado en: {ruta_salida}")


archivo_csv = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados\promedio_final.csv"  
carpeta_resultados = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados"
Setpoint = 20 

Setpoint_Max(archivo_csv, Setpoint ,carpeta_resultados)


### maximos partiendo de setpoint para toda la carpeta de muestras 

In [ ]:
def procesar_archivos_en_carpeta(carpeta_entrada, carpeta_resultados,Setpoint ):
    archivos_csv = [f for f in os.listdir(carpeta_entrada) if f.endswith('.csv')]
    
    for archivo in archivos_csv:
        archivo_csv = os.path.join(carpeta_entrada, archivo)
        data = pd.read_csv(archivo_csv)
        encabezados = data.columns

        promedio_segunda_columna = data.iloc[:, 1].mean()
        setpoint = promedio_segunda_columna + Setpoint 

        valores_superan_setpoint = data[data.iloc[:, 1] > setpoint]

        nombre_original_sin_extension = os.path.splitext(os.path.basename(archivo_csv))[0]
        nombre_salida = f"{nombre_original_sin_extension}_Setpoint_Max.csv"

        if not os.path.exists(carpeta_resultados):
            os.makedirs(carpeta_resultados)

        ruta_salida = os.path.join(carpeta_resultados, nombre_salida)
        valores_superan_setpoint.to_csv(ruta_salida, index=False)

        print(f"Valor promedio dB {archivo}: {promedio_segunda_columna} dB")
        print(f"Valor Setpoint para {archivo}: {setpoint} dB")
        print(f"El archivo de resultados se ha guardado en: {ruta_salida}")

carpeta_entrada = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras"
carpeta_resultados = r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados"
Setpoint = 20 

procesar_archivos_en_carpeta(carpeta_entrada, carpeta_resultados,Setpoint)

# pruebas

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Cargar datos de los dos archivos CSV
data1 = pd.read_csv(r'C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Muestras\Muestra_1.csv')
data2 = pd.read_csv(r"C:\Users\dfgom\OneDrive\Escritorio\USRP\RFI_Captura\Salida\Resultados\Muestra_1_Setpoint_Max.csv")

encabezado1 = data1.columns
encabezado2 = data2.columns

# Extraer las columnas de interés de cada archivo (por ejemplo, 'x' y 'y')
x1, y1 = data1[encabezado1[0]], data1[encabezado1[1]]
x2, y2 = data2[encabezado2[0]], data2[encabezado1[1]]

fig, ax = plt.subplots()

# Graficar los datos del primer archivo CSV (por ejemplo, como una línea)
ax.plot(x1, y1, label='Archivo 1 - Línea')

# Graficar los datos del segundo archivo CSV (por ejemplo, como puntos)
scatter = ax.scatter(x2, y2, color='red', marker='o', label='Archivo 2 - Puntos')

# Personalizar etiquetas
ax.set_xlabel(encabezado1[0])
ax.set_ylabel(encabezado1[1])

# Agregar una leyenda
ax.legend()

# Mostrar el gráfico
plt.show()